### Automate Your Billing Workflow with the Stripe Agent Toolkit and OpenAI Agents

Running a service business means juggling two things at once: doing the work and getting paid for it. For a solo electrician or small field service operation, that second part — turning a completed job into a sent invoice — can mean switching between tools, manually entering line items, and chasing down customer records. This cookbook demonstrates how to automate that entire workflow using the OpenAI Agents SDK and the Stripe Agent Toolkit. Rather than building a single monolithic agent, we'll take a multi-agent approach — designing a small team of specialized agents, each responsible for a distinct part of the workflow, coordinated by an orchestrator that routes work between them automatically.
Our workflow handles three core tasks end-to-end:

* **Account lookup**: Identifying the customer by email.
* **Quote generation**: Building an itemized quote from a service catalog.
* **Invoicing**: Creating and finalizing a billable invoice in Stripe.

The [Stripe Agent Toolkit](https://docs.stripe.com/agents) is central to this workflow. It provides a set of pre-built, agent-ready tools that connect directly to the Stripe API — covering customers, products, prices, and invoices — without requiring you to write any integration code. Paired with a restricted API key, you can enforce least-privilege permissions so the agent only has access to exactly what it needs.

By the end of this cookbook, a single natural language message like *"Create an invoice for john@example.com for installing a ceiling fan, which took 2 hours"* will trigger a fully automated pipeline that looks up the customer, prices the job, and returns a finalized Stripe invoice URL — ready to send.



### Prerequisites

Before getting started, you'll need:

* **A Stripe account** in test mode with a [restricted API key](https://docs.stripe.com/keys-best-practices#limit-access) scoped to customers, products, prices, and invoices — create one at [dashboard.stripe.com/apikeys](dashboard.stripe.com/apikeys)
* **An OpenAI account** with an API key — sign up at [platform.openai.com](https://platform.openai.com/)

Store both keys securely. In this cookbook we'll load them from Google Colab's secret store, but in a production environment they should live in a secrets manager or environment variable - never hardcoded.

### Installation & Setup

First, we'll install the three packages needed to build this workflow.

* `stripe-agent-toolkit` is Stripe's official library that provides pre-built, agent-ready tools for interacting with the Stripe API.
* `openai-agents` is the OpenAI agent framework that handles tool orchestration and the agent loop.
* `openai` is the underlying SDK for communicating with the model.

In [3]:
# Install OpenAI agent framework & Stripe Agent Toolkit
!pip install stripe-agent-toolkit openai-agents openai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.1/388.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.6 MB/s eta 0:00:00


Next, we'll import the packages providing access to Colab's secure secret store, the OpenAI Agents SDK, and libraries for data modeling and type validation.


In [54]:
# Load Collab ENV Variables
from google.colab import userdata

# OpenAI & Agents Framework
from agents import function_tool, set_default_openai_key, Agent, Runner, trace, set_tracing_disabled
from openai import AsyncOpenAI

# Data Typex
from typing import Any, Dict, List, Optional
from pydantic import BaseModel, Field

# Stripe Toolkit w/ OpenAI Toolkit (Connects to Stripes MCP server)
from stripe_agent_toolkit.openai.toolkit import create_stripe_agent_toolkit

Use a [Stripe Restricted API key](https://docs.stripe.com/keys-best-practices#limit-access) with the Agent toolkit to enforce least-privilege permissions, limiting the agent to only the specific reads/writes it needs (e.g., customers/products/prices/invoices). Note that Restricted API keys should be stored securely in your web or mobile app's server-side code (such as in an environment variable or credential management system) to call Stripe APIs.

Additionally, you can set your OpenAI API key for use by the agents via the `set_default_openai_key` function.

In [55]:
# LOAD API Keys
STRIPE_API_KEY = userdata.get('STRIPE_SANDBOX_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Set OpenAI API Key
if OPENAI_API_KEY:
  set_default_openai_key(f"{OPENAI_API_KEY}")
  set_tracing_disabled(True)

### Product Catalog & Account Lookup Services

Here we set up a mock in-memory service catalog representing the electrician's pricing data as a simple dictionary of flat-rate services each with an ID, description, price, and currency. In a real-world setting, this would be replaced by a live database or MCP server exposing the catalog through a set of permissioned tools, ensuring the agent only has access to the data it needs and that pricing stays consistent with the source of truth.


In [36]:
### CONSTANTS ###
HOURLY_RATE = 175.00
DEFAULT_CURRENCY = "usd"

### MOCK DB ###
SERVICE_DB: Dict[str, Dict[str, Any]] = {
    "diagnostic": {
        "id": "diagnostic",
        "description": "Service call / diagnostic (up to 30 min)",
        "unit_amount": 129.00,
        "currency": DEFAULT_CURRENCY,
    },
    "outlet_replace": {
        "id": "svc_outlet_replace",
        "description": "Replace standard outlet (receptacle)",
        "unit_amount": 165.00,
        "currency": DEFAULT_CURRENCY,
    },
    "gfci_install": {
        "id": "gfci_install",
        "description": "Install GFCI outlet",
        "unit_amount": 245.00,
        "currency": DEFAULT_CURRENCY,
    },
    "fan_install": {
        "id": "svc_fan_install",
        "description": "Install ceiling fan (existing wiring)",
        "unit_amount": 385.00,
        "currency": DEFAULT_CURRENCY,
    },
}

With the catalog in place, we define four tools that give our agents their core capabilities.

* `lookup_account` returns a dummy account based on a user ID provided by the user.
* `list_services` returns the full service catalog.
* `lookup_service` retrieves pricing for a specific service by ID.
* `calculate_labor_cost` computes a labor line item for time-and-materials work.

The `@function_tool` decorator automatically converts the function signature and docstring into a JSON schema that the agent framework can use to understand when and how to invoke it.

In [69]:
### INPUT VALIDATION ###
class LaborCostInput(BaseModel):
    hours: float = Field(..., gt=0, description="Number of billable hours worked.")
    hourly_rate: float = Field(
        HOURLY_RATE,
        gt=0,
        description=f"Hourly labor rate in USD. Defaults to ${HOURLY_RATE}/hr.",
    )


class LookupServiceInput(BaseModel):
    service_id: str = Field(
        ...,
        description=(
            "The unique ID of the flat-rate service to look up. "
            "Call list_services first if you are unsure of the exact ID."
        ),
    )

### Tool Calls ###
@function_tool # Decorater function to convert function into schema for agent.
async def lookup_account(email: str) -> str:
  """ Returns a dummy account based on email sourced from DB """
  return {
      "id": "uid_123456",
      "name": "John Doe",
      "email": email,
      "billing_address": {
          "street": "324 Idaho Ave",
          "city": "Santa Monica",
          "state": "CA",
          "zip": "90403"
      },
  }

@function_tool
async def calculate_labor_cost(input: LaborCostInput) -> Dict[str, Any]:
    """
    Calculate a labor line item based on hours worked and an hourly rate.
    Use this for time-and-materials work not covered by a flat-rate service,
    or when the user asks about the cost of a specific number of labor hours.
    Returns a line-item dict with description, unit_amount, and currency.
    """
    total = round(input.hours * input.hourly_rate, 2)
    return {
        "description": f"Labor ({input.hours} hours @ ${input.hourly_rate}/hr)",
        "unit_amount": total,
        "currency": DEFAULT_CURRENCY,
    }


@function_tool
async def lookup_service(input: LookupServiceInput) -> Dict[str, Any]:
    """
    Look up pricing and details for a single flat-rate service by its service_id.
    Call list_services first if you don't know the exact service_id.
    Returns a dict with description, unit_amount, and currency.
    Returns an error dict (with available_ids) if the service_id is not found.
    """
    service = SERVICE_DB.get(input.service_id)
    if not service:
        return {
            "error": f"No service found with id '{input.service_id}'.",
            "available_ids": list(SERVICE_DB.keys()),
        }
    return service


@function_tool
async def list_services() -> List[Dict[str, Any]]:
    """
    Return all available flat-rate electrician services with their IDs, descriptions,
    and prices. Call this when the user asks what services are offered, or when you
    need to find the correct service_id before calling lookup_service.
    """
    return list(SERVICE_DB.values())

### Stripe Toolkit Setup

Here we initialize the [Stripe Agent Toolkit](https://docs.stripe.com/agents) with a restricted API key, explicitly scoping permissions to only the four resources our workflow needs: `customers`, `products`, `prices`, and `invoices`. Any Stripe resource not listed is automatically blocked, ensuring the agent operates under least-privilege — it can't touch anything outside of what the invoicing workflow requires.


In [51]:
\



async def init_stripe_agent_toolkit():
  # Initialize Toolkit
  toolkit = await create_stripe_agent_toolkit(
      secret_key=STRIPE_API_KEY,
        configuration={
          "actions": {
            "products": {"create": True, "read": True, "update": True},
            "prices": {"create": True, "read": True, "update": True},
            "invoices": {"create": True, "read": True, "update": True, "finalize": True},
            "customers": {"create": True, "read": True, "update": True},
          }
        }
  )
  return toolkit

### Agent Instructions

With our tools and Stripe toolkit in place, we can now define the instruction prompts that govern each agent's behavior. Each prompt defines the agent's role, the rules it must follow when invoking its tools, and the format it should return results in. Well-written instructions are what separate a reliable agent from one that hallucinates values or stops mid-workflow waiting for input it already has.

We define four prompts in total — one for each specialist agent and one for the orchestrator that coordinates between them.

In [84]:
account_agent_instructions = """
  You are an account support agent. Your job is to retrieve and present account details for a given email address.

  <instructions>
  1) Always call `lookup_account` when the user asks for information about a user. Do not guess or invent values.
  2) If the user did not provide an email, ask a single clarifying question requesting it.
  3) If the user asks for data not included in the tool response, state that it is unavailable and offer what you can provide.
  </instructions>

  <output>
  Return a JSON object only. Do not include additional commentary outside the JSON.
  </output>
"""

### PRODUCT CATALOG AGENT INSTRUCTIONS ###
product_catalog_agent_instructions = """
  You are a pricing and product catalog assistant. Your job is to help customers understand what services are offered, what they cost,
  and to build accurate, itemized quotes for their electrical work.

  <instructions>
  1. Always retrieve live data from your tools. Never quote prices from memory or make them up.
  2. When a customer asks about available services, present them as a clean catalog with
    descriptions and flat-rate prices.
  3. When building a quote, look up each relevant service and include a labor line item if
    the job involves time-and-materials work. Sum all line items into a subtotal.
  4. If a requested service is not in the catalog, tell the customer it isn't currently offered
    and present what is available.
  5. If a customer asks about labor, confirm the number of hours before calculating. If they
    haven't provided hours, ask a single clarifying question.
  6. Only present pricing information sourced from tool responses. If a detail isn't available
    in the catalog, say so and offer what you can.
  </instructions>

  <output_format>
  For quotes, return a JSON object only:

  {
    "quote": {
      "line_items": [
        { "description": "...", "unit_amount": 0.00, "currency": "usd" }
      ],
      "subtotal": 0.00,
      "currency": "usd"
    }
  }

  For general catalog questions, respond in clear and friendly prose.
  </output_format>
"""

### STRIPE AGENT INSTRUCTIONS ###
stripe_agent_instructions = """
  You are a helpful payment assistant who helps electricians manage customers and payments through Stripe to support their business needs.

  <instructions>
  When an electrician enters a customer (email & name) and fulfilled service item, please:
    1. Look up that customer to see if they already exist in Stripe (by email address).
      - If they do, retrieve the customerID.
    2. Create a product & price in Stripe based on service descriptions & prices provided by the electricians.
      - If the product & price already exist, retrieve the & priceID.
      - If the product exists but it's a different price, create a new price attached that product.
    3. After you've retrieved the customer, create a billiable invoice in Stripe for the customer
      - Please create all product & price data inline, rather than calling Price tool.
      - Set the invoice to manual collection, with an immediate due date.
    4. Add each line item from the quote as a SEPARATE invoice line item. Do not combine or
        aggregate line items. Each item in the quote's line_items array must become its own
        distinct invoice line item with its own description and amount.
    5. Finalize the invoice.
    6.. Return hosted_invoice_url as the 'invoiceURL'.
  </instructions>

  <output_format>
  Once complete, please return the the following information in JSON format with no additional information:

  {
    "returning_customer: "boolean",
    "customerID": "string",
    "invoiceID": "string"
    "invoiceAmount: "float"
    "invoiceURL": "string"
    "prices": [
      {
        "product_exists: "boolean",
        "price_exists: "boolean",
        "priceID": "string",
        "productID": "string",
        "productDescription": "string",
        "priceAmount": "float"
    ]
  }
  <output_format>
"""

### ORCHESTRATOR AGENT ###
orchestrator_instructions = """
You are the primary orchestration agent for an electrician service business. Your job is to
understand what the customer needs and route work to the appropriate specialist agents to
complete it end-to-end — autonomously and without stopping for confirmation between steps.

<agents>
You have access to the following specialist agents. Delegate to them — do not attempt to
perform their work yourself.

- `account_agent` — Handles customer account lookups by email address. Use this first whenever
  a customer is referenced to establish who they are before proceeding.

- `product_catalog_agent` — Handles service discovery and quote generation. Use this when
  the customer asks about available services, pricing, or needs an itemized quote built.

- `stripe_agent` — Handles all payment operations including customer creation, invoice
  generation, and finalization. Use this after a quote has been built to issue and finalize
  the invoice.
</agents>

<instructions>
0. The original user message contains all information needed to complete the full workflow.
   Extract and carry forward all relevant details from it — email, service description, and
   labor hours — before making any agent handoff. Do not ask for information that was already
   provided in the original request.
1. Always start by establishing customer identity. If an email address is provided, call
   `account_agent` first to retrieve their account details before routing to any other agent.
2. If the customer is asking about services or pricing, delegate to `product_catalog_agent`.
3. For invoice requests, run the full workflow sequentially without stopping:
   account_agent → product_catalog_agent → stripe_agent.
4. When handing off between agents, carry forward all relevant context — email, name,
   confirmed services, and quoted amounts — so no agent has to ask for information that
   has already been established.
5. Do not perform lookups, pricing calculations, or payment operations yourself. Always
   delegate to the appropriate specialist agent.
6. Never stop after a single agent handoff if the original request requires multiple steps.
   Complete the full workflow autonomously and return the final consolidated output only.
7. You are an orchestrator, not a responder. Do not summarize intermediate results or ask
   the user for confirmation between steps unless a required input is genuinely missing.
8. If any agent returns an error, surface it clearly and do not proceed to downstream steps
   until it is resolved.
</instructions>

<output_format>
For multi-step workflows that result in a finalized invoice, return a single JSON object
consolidating the outputs of all agents:

{
  "account": {
    "userId": "...",
    "name": "...",
    "email": "..."
  },
  "quote": {
    "line_items": [
      { "description": "...", "unit_amount": 0.00, "currency": "usd" }
    ],
    "subtotal": 0.00,
    "currency": "usd"
  },
  "invoice": {
    "returning_customer": true,
    "customerID": "...",
    "invoiceID": "...",
    "invoiceAmount": 0.00,
    "invoiceURL": "..."
  }
}

For single-step requests such as a catalog lookup or account check, respond in clear prose
or delegate the output format to the relevant specialist agent.
</output_format>
"""

### Defining the Agents


With our instructions defined, we can wire everything together into a runnable multi-agent system. Each specialist agent is initialized with its instruction prompt and the tools it needs to do its job. The orchestrator is then configured with all three specialist agents exposed as tools via `as_tool()` — this is an important distinction from using `handoffs`. While `handoffs` transfer control permanently to a sub-agent, `as_tool()` keeps the orchestrator in the loop so it can invoke each agent sequentially, carry context forward between steps, and return a single consolidated response at the end.


In [82]:
async def main(message: str):
    # Init Stripe toolkit
    stripe_toolkit = await init_stripe_agent_toolkit()
    # Account Lookup Agent
    account_agent = Agent(
        name="Account Agent",
        instructions=account_agent_instructions,
        tools=[lookup_account],
    )
    # Product Catalog Agent
    product_catalog_agent = Agent(
        name="Product Catalog Agent",
        instructions=product_catalog_agent_instructions,
        tools=[lookup_service, list_services, calculate_labor_cost],
    )
    # Stripe Agent
    stripe_agent = Agent(
        name="Stripe Agent",
        instructions=stripe_agent_instructions,
        tools=stripe_toolkit.get_tools(),

    )

    # Orchestrator Agent
    orchestrator_agent = Agent(
        name="Orchestrator Agent",
        instructions=orchestrator_instructions,
        tools=[
            account_agent.as_tool(
                tool_name="account_agent",
                tool_description="Look up customer account details by email address."
            ),
            product_catalog_agent.as_tool(
                tool_name="product_catalog_agent",
                tool_description="Look up services, pricing, and build itemized quotes."
            ),
            stripe_agent.as_tool(
                tool_name="stripe_agent",
                tool_description="Create customers, generate invoices, and finalize payments in Stripe."
            ),
      ]
    )

    result = await Runner.run(orchestrator_agent, message)
    print(result.final_output)

### Running the Workflow

With everything wired together, we can now test the full system against three scenarios that exercise each part of the workflow independently and in combination.

The first query tests the account agent in isolation, confirming it can look up a customer record by email. The second tests the product catalog agent, verifying it returns a clean, accurate service listing. The third — and most important — triggers the full end-to-end pipeline: account lookup, quote generation, and Stripe invoice creation, all from a single natural language message.

In [88]:
# Test Customer Lookup
await main("Please lookup john@example.com")
print("--------")
# Test Product Lookup
await main("What services are available to sell?")
print("--------")
# Test Quote Generation
await main("Please create an invoice for john@example.com for installing a ceiling fan, which took 2 hours",)

{
  "account": {
    "userId": "uid_123456",
    "name": "John Doe",
    "email": "john@example.com",
    "billing_address": {
      "street": "324 Idaho Ave",
      "city": "Santa Monica",
      "state": "CA",
      "zip": "90403"
    }
  }
}
--------
The following flat-rate electrical services are currently available:

1. Service call / diagnostic (up to 30 min) – $129.00
2. Replace standard outlet (receptacle) – $165.00
3. Install GFCI outlet – $245.00
4. Install ceiling fan (existing wiring) – $385.00

All prices are in USD. If you need more details on any service or want an itemized quote, just let me know!
--------
{
  "account": {
    "userId": "uid_123456",
    "name": "John Doe",
    "email": "john@example.com"
  },
  "quote": {
    "line_items": [
      { "description": "Install ceiling fan (existing wiring)", "unit_amount": 385.00, "currency": "usd" },
      { "description": "Labor (2.0 hours @ $175.0/hr)", "unit_amount": 350.00, "currency": "usd" }
    ],
    "subtotal": 73

### Conclusion

In this cookbook we've demonstrated how the OpenAI Agents SDK and Stripe Agent Toolkit can work together to automate a complete billing workflow — from customer lookup to finalized invoice — triggered by a single natural language message. [The Stripe Agent Toolkit](https://docs.stripe.com/agents) does the heavy lifting on the payments side, providing pre-built, agent-ready tools for the full invoicing lifecycle without requiring any custom Stripe integration code. Paired with a restricted API key and a focused set of permissions, it's a pattern that's straightforward to extend — swap in additional Stripe resources, grow the service catalog, or add new specialist agents as your business needs evolve.
